In [ ]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
import json
from PIL import ImageDraw
from copy import copy
from PIL import ImageFont
import time
font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 15)
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
model = K.models.load_model('../../model/resnet50_preprocessing_checkpoints.h5')

In [ ]:
filepath = glob('../../data/task_0421_p20_backup_2023_05_10_13_31_48/data/*.jpg')
save_path= [f.replace('/task_0421_p20_backup_2023_05_10_13_31_48/data', '/video/image1') for f in filepath]


In [ ]:

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


for i in range(len(filepath)):
    start = time.time()
    image=Image.open(filepath[i])
    dst_image=copy(image)
    image=image.crop((196,128,508,640))
    image=image.resize((156,256))
    image=expand2square(image,(0,0,0))
    image=np.array(image).reshape(1,256,256,3)/255
    pred=model.predict(image)
    draw = ImageDraw.Draw(dst_image)
    if pred>0.449:
        text="standardFrame"+str(pred)
        fill1="green"
        draw.rectangle((0,0,dst_image.size[0],dst_image.size[1]), outline=(0,255,0), width = 3)
    else :
        text="Not standardFrame"+str(pred)
        fill1="red"
        draw.rectangle((0,0,dst_image.size[0],dst_image.size[1]), outline=(255,0,0), width = 3)
        
    draw.text((10,10), text, font=font, fill=fill1)
    
    end = time.time()
    draw.text((10,25), f"{end - start:.5f} sec", font=font, fill=fill1)
    draw = ImageDraw.Draw(dst_image)
    dst_image.save(save_path[i])

In [ ]:
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)
print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)